In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# Stochastic Lie group dynamics

In [ ]:
# SO(3)
from src.groups.SON import *
G = SON(3)
print(G)

# SO(3) acts on S^2
from src.manifolds.S2 import *
M = S2()
print(M)

from src.plotting import *

In [ ]:
# setup for testing different versions of stochastic dynamics
q = np.array([1e-6,0,0])
g = G.psif(q)
v = np.array([0,1,1])

from src.group import invariant_metric
invariant_metric.initialize(G)
p = G.sharppsif(q,v)
mu = G.sharpVf(v)
print(p)
print(mu)

from src.group import energy
energy.initialize(G)

In [ ]:
# Brownian motion
from src.stochastics import Brownian_inv
Brownian_inv.initialize(G)

n_steps.set_value(1000)

# srng.seed(422)
srng.seed(407)
(ts,gsv) = G.Brownian_invf(g,dWsf(G.dim.eval()))
newfig()
G.plotg(gsv,linewidth=0.1,alpha=0.1)
plt.show()
#plt.savefig('stocso3.pdf')

# on S2
newfig()
M.plot()
x = np.array([0,0,1])
M.plotx(M.actsf(gsv.transpose((1,2,0)),x).T)
plt.show()
#plt.savefig('stocso32.pdf')

In [ ]:
# Brownian processes
from src.stochastics import Brownian_process
Brownian_process.initialize(G)

(ts,gsv) = G.Brownian_processf(g,dWsf(G.dim.eval()))
newfig()
G.plotg(gsv)
plt.show()

In [ ]:
# Stochastic coadjoint motion
from src.stochastics import stochastic_coadjoint
stochastic_coadjoint.initialize(G)

(ts,musv) = G.stochastic_coadjointf(mu,dWsf(G.dim.eval()))
(ts,gsv) = G.stochastic_coadjointrecf(g,musv)
newfig()
G.plotg(gsv,color_intensity=1,alpha=0.1)
plt.show()
#plt.savefig('coadgeo.pdf')

In [ ]:
# Delyon/Hu guided process
from src.stochastics.guided_process import *

# guide function
# phi = lambda g,v: -(g-v)
phi = lambda g,v: T.tensordot(G.inv(G.sigma),G.LAtoV(G.log(G.invtrns(G.inv(g),v))),(1,0))
g0 = G.element()
(dlog_likelihood,dlog_likelihoodf,Brownian_inv_guided,Brownian_inv_guidedf) = get_guided_likelihood(G,G.sde_Brownian_inv,phi,lambda g: G.sigma, g0, (), A=G.gG, integration='stratonovich')

(ts,gs,log_likelihood,log_varphi) = Brownian_inv_guidedf(g,G.psif(v),dWsf(G.dim.eval()))
print("log likelihood: ", log_likelihood[-1], ", log varphi: ", log_varphi[-1])

newfig()
w = G.psif(v)
G.plotg(gs)
G.plotg(w,color='k')
plt.show()

# on S2
newfig()
M.plot()
M.plotx(M.actsf(gs.transpose((1,2,0)),x).T)
M.plotx(M.actf(w,x),color='k')
plt.show()